<a href="https://colab.research.google.com/github/shanmcm/Sentiment-Analysis-Amazon-Musical-Instruments-Reviews/blob/main/Amazon_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import torch
import torch.nn as nn
import os
import random
import pandas as pd
from torchtext import data
import pickle5 as pickle

In [12]:
data_org = { 'a':'A', 'b':2, 'c':3.0 } 
print('DATA:', data_org)

DATA: {'a': 'A', 'b': 2, 'c': 3.0}


In [13]:
data_string = pickle.dumps(data_org)
print('PICKLE:', data_string )

PICKLE: b'\x80\x04\x95 \x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\x01a\x94\x8c\x01A\x94\x8c\x01b\x94K\x02\x8c\x01c\x94G@\x08\x00\x00\x00\x00\x00\x00u.'


In [14]:
with open('pickled_data_file.pkl', 'wb') as fid:
  pickle.dump(data_org, fid)

In [15]:
with open('pickled_data_file.pkl', 'rb') as fid:
  data3 = pickle.load(fid)

In [16]:
print('Data Before Write:', data_org)
print('Data After  Read :', data3)
print('EQUAL?:', (data_org == data3))

Data Before Write: {'a': 'A', 'b': 2, 'c': 3.0}
Data After  Read : {'a': 'A', 'b': 2, 'c': 3.0}
EQUAL?: True


In [17]:
with open('amazonDataset.pkl', 'rb') as fid:
  data3 = pickle.load(fid)

AttributeError: ignored

In [8]:
data_string = pickle.dumps(data_org)
print('PICKLE:', data_string )

PICKLE: b'\x80\x04\x95 \x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\x01a\x94\x8c\x01A\x94\x8c\x01b\x94K\x02\x8c\x01c\x94G@\x08\x00\x00\x00\x00\x00\x00u.'


In [6]:
with open('amazonDataset.pkl', 'rb') as fid:
  data3 = pickle.load(fid)

AttributeError: ignored

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Parte Lucia

In [ ]:
'''
Qui c'è un bozza sulla parte iniziale (settaggio parametri, creazione train test ecc)
Faccio riferimento a una funzione cleanup_text che va riempita in base al preprocessing
'''
import params
from lstm_cell import LSTMCell

def cleanup_text(texts):
  cleaned_text = []
  for text in texts:
      # remove punctuation, ecc....
      text = re.sub('[!#?,.:";]', ' ', text)
     
      cleaned_text.append(text)
  return cleaned_text


char_based = True
if char_based:
    tokenizer = lambda s: list(s) # char-based
else:
    tokenizer = lambda s: s.split() # word-based


# hyper-parameters:
lr = params.LR
batch_size = params.BATCH_SIZE
dropout_rate = params.DROPOUT_RATE
#dropout_keep_prob = 0.5
embedding_size = 300
max_document_length = 100  # each sentence has until 100 words
dev_size = 0.8 # split percentage to train\validation data
max_size = 5000 # maximum vocabulary size
seed = params.SEED
num_classes = params.NUM_CLASSES # output dimension of LSTM
num_hidden_nodes = 93
hidden_dim2 = 128
num_layers = 2  # LSTM layers
bi_directional = True
epochs = params.NUM_EPOCHS

Text = data.Field(preprocessing=cleanup_text, tokenize=tokenizer, batch_first=True, include_lengths=True, fix_length=max_document_length) # fix_length - make the sentences padded in the same lengths for all the batches
Label = data.Field(sequential=False, use_vocab=False, pad_token=None, unk_token=None)

# All files:
fields = [('text', Text), ('labels', Label)]

data_type = "token" # or: "morph"

train_data, test_data = data.TabularDataset.splits(
 # https://pytorch.org/text/_modules/torchtext/data/dataset.html
)

train_data, valid_data = train_data.split(split_ratio=dev_size, random_state=random.seed(seed))

# Build_vocab : It will first create a dictionary mapping all the unique words present in the train_data to an
# index and then after it will use word embedding (random, Glove etc.) to map the index to the corresponding word embedding.
Text.build_vocab(train_data, max_size=max_size)
Label.build_vocab(train_data)
vocab_size = len(Text.vocab)

#  BucketIterator : Defines an iterator that batches examples of similar lengths together to minimize the amount of padding needed.
# by setting sort_within_batch = True.
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data),
    batch_size = batch_size,
    sort_key = lambda x: len(x.text), # Sort the batches by text length size
    sort_within_batch = True,
    device = device)

Parte Shannon

In [ ]:
to_train = True
pad_index = Text.vocab.stoi[Text.pad_token]

# Build the model
lstm_model = SentimentAnalysis(vocab_size, embedding_size, num_hidden_nodes, hidden_dim2 , num_classes, num_layers,
                bi_directional, dropout_keep_prob, pad_index) #modificato LSTM con SentimentAnalysis (nome rete)

# optimization algorithm
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=lr)

# train and validate
if (to_train):

    for epoch in range(epochs):

        # training 
        epoch_loss = 0
        epoch_acc = 0
        for batch in train_iterator:
            optimizer.zero_grad()
            # retrieve text and no. of words
            text, text_lengths = batch.text

            predictions = lstm_model(text, text_lengths) # batch_size, hidden_dim, vocab_size, window, dropout_rate
            loss = 0.5*nn.CrossEntropyLoss(predictions, batch.labels.squeeze())+ 0.5*nn.MSELoss(predictions, batch.labels.squeeze())

            winners = predictions.argmax(dim=1)
            corrects = (winners == batch.labels)
            accuracy = corrects.sum().float() / float(batch.labels.size(0))

            # perform backpropagation
            loss.backward()

            optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += accuracy.item()
            
        train_loss, train_acc = epoch_loss / len(train_iterator), epoch_acc / len(train_iterator)
        
        # validation
        epoch_loss = 0
        epoch_acc = 0

        lstm_model.eval()

        with torch.no_grad():
            for batch in valid_iterator:
                text, text_lengths = batch.text

                predictions = lstm_model(text, text_lengths).squeeze(1)

                loss = 0.5*nn.CrossEntropyLoss(predictions, batch.labels)+ 0.5*nn.MSELoss(predictions, batch.labels)

                winners = predictions.argmax(dim=1)
                corrects = (winners == batch.labels)
                accuracy = corrects.sum().float() / float(batch.labels.size(0))

                epoch_loss += loss.item()
                epoch_acc += accuracy.item()

        valid_loss, valid_acc =  epoch_loss / len(valid_iterator), epoch_acc / len(valid_iterator)


        # save best model
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(lstm_model.state_dict(), 'saved_weights_BiLSTM.pt')

        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')


# load weights and make predictions
lstm_model.load_state_dict(torch.load("saved_weights_BiLSTM.pt"))
epoch_loss = 0
epoch_acc = 0

lstm_model.eval()

with torch.no_grad():
    for batch in test_iterator:
        text, text_lengths = batch.text

        predictions = lstm_model(text, text_lengths).squeeze(1)

        loss = 0.5*nn.CrossEntropyLoss(predictions, batch.labels)+ 0.5*nn.MSELoss(predictions, batch.labels)

        winners = predictions.argmax(dim=1)
        corrects = (winners == batch.labels)
        accuracy = corrects.sum().float() / float(batch.labels.size(0))

        epoch_loss += loss.item()
        epoch_acc += accuracy.item()

test_loss, test_acc =  epoch_loss / len(test_iterator), epoch_acc / len(test_iterator)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%')

In [ ]:
!git status

In [ ]:
!git add --all

In [ ]:
!git config --global user.email "s289958@studenti.polito.it" # mettere email del proprio github account

In [ ]:
!git commit -m"Added utils.py with useful functions"

In [ ]:
!git push